<a href="https://colab.research.google.com/github/solanods/data_analysis/blob/main/alteracoes_rmops3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

from google.colab import drive
drive.mount('/content/drive')

from datetime import date

from re import search

import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


In [ ]:
# Carregamento dos dados
arquivo_antigo = '/content/PRMS_01_JAN_2024_terca.xlsm'
arquivo_recent = '/content/PRMS_01_JAN_2024.xlsm'

df_original = pd.read_excel(arquivo_antigo, sheet_name='RMOP')
df_alterado = pd.read_excel(arquivo_recent, sheet_name='RMOP')


colunas_alvo = list(range(327, 359))
df_escalas = pd.read_excel(arquivo_recent, sheet_name='DADOS', usecols=colunas_alvo)
df_escalas = df_escalas.loc[5:200]

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# Funcao limpeza de dados

def tratarDataframe(df):

  # Selecionar colunas de interesse
  df = df.iloc[:,[3,5,6,7,8,11,16,17,18,19]]

  # Renomear colunas
  df = df.rename({'Unnamed: 3': 'DIA',
                  'Unnamed: 5': 'PREVISTO',
                  'Unnamed: 6': 'ESCALA',
                  'Unnamed: 7': 'TOTVS',
                  'Unnamed: 8': 'JORNADA',
                  'Unnamed: 11': 'ENTRADA',
                  'Unnamed: 16': 'CODIGO',
                  'Unnamed: 17': 'DESCRICAO',
                  'Unnamed: 18': 'OBSERVACOES',
                  'Unnamed: 19': 'RE_SGT'}, axis = 1)

  # Remover nan
  df = df.fillna('-')

  # Obter lista com nome de todos operadores
  operadores = []
  for linha in df.index:
    if search ('[A-Z]+', str(df['DIA'][linha])):
      operadores.append(df['DIA'][linha])

  operadores = [operador for operador in operadores if operador.isupper()]

  # Extrair rmop
  i = 3
  j =38
  R = []
  df['NOME'] = ''
  df['RE'] = ''
  for operador in operadores:
    df['NOME'].loc[i:j] = operador
    rmop_operador = df.loc[i:j]
    R.append(rmop_operador)
    df['RE'].loc[i:j] = df['ENTRADA'].loc[i:j].iloc[0]
    i+=66
    j+=66

  return df

In [ ]:
df_original = tratarDataframe(df_original)
df_alterado = tratarDataframe(df_alterado)

In [ ]:
# Criar novo df somente com os índices modificados, ou seja, as diferenças entre ambos
rmops_alterado = pd.concat([df_original,df_alterado]).drop_duplicates(keep=False)

In [ ]:
# Resultado mostra os indices originais e alterados em sequencia
rmops_alterado

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,NOME,RE
621,20,TRAB,J5,13-M,14N,20:40:00,-,-,-,-,ANDREI,3552
627,26,TRAB,J5,13-M,03T,13:30:00,-,-,-,-,ANDREI,3552
1018,21,TRAB,MA,3-C,TT1,13:15:00,-,-,-,-,BARCELLOS,542
1277,16,TRAB,EB,2-B,NMR,21:30:00,-,-,-,-,BENHUR DA SILVA,1853
1808,19,FOL,M1,7-G,FOL,,D,Disponível para HE,-,-,CHAVES,3312
1937,16,TRAB,G3,17-Q,02N,21:00:00,-,-,-,-,CLAUDETE,2765
3589,18,FOL,Q6,17-Q,13T,14:20:00,C129,Troca Jorn,POR 21,2934,FROZZA,2451
5105,16,TRAB,G2,17-Q,01N,21:15:00,-,-,-,-,JULIO,1808
5175,20,TRAB,QA,3-C,TPT,13:15:00,-,-,-,-,JUNIOR,2426
5176,21,TRAB,QA,3-C,TPT,13:15:00,-,-,-,-,JUNIOR,2426


In [ ]:
# Criar campo data
data_atual = date.today()
rmops_alterado['DATA_COLETA_DADO'] = data_atual

In [ ]:
# Salvar no google drive
rmops_alterado.to_excel(f'/content/drive/MyDrive/alteracoesrmop{data_atual}.xlsx')

In [ ]:
# Numero de alteracoes
num_alteracoes = len(rmops_alterado.index)/2

In [ ]:
# String para arquivar
dados = str(data_atual) + '   N°_ALTERACOES: ' + str(int(num_alteracoes))

In [ ]:
dados

'2024-01-17   N°_ALTERACOES: 24'

In [ ]:
# Salvar em arquivo texto
file = open('/content/drive/MyDrive/alteracoes_rmop_contador.txt', 'a')
file.write(dados + '\n')
file.close()

In [ ]:
# Verificar entre jornadas

for i in df_alterado.index:
    if search ('\d{2}T', str(df_alterado['JORNADA'][i])):
      if search ('\d{2}M', str(df_alterado['JORNADA'][i+1])):

        print('VERIFICAR INTERVALO DE 11 HORAS ENTRE JORNADAS\n')

        print('Dia',
              df_alterado['DIA'][i],
              df_alterado['NOME'][i],
              df_alterado['JORNADA'][i],
              df_alterado['ENTRADA'][i])

        print('Dia',
              df_alterado['DIA'][i+1],
              df_alterado['NOME'][i+1],
              df_alterado['JORNADA'][i+1],
              df_alterado['ENTRADA'][i+1])

VERIFICAR INTERVALO DE 11 HORAS ENTRE JORNADAS

Dia 24 MATHEUS 14T 12:00:00
Dia 25 MATHEUS 11M 06:30:00
VERIFICAR INTERVALO DE 11 HORAS ENTRE JORNADAS

Dia 10 RENAN 14T 12:00:00
Dia 11 RENAN 11M 06:30:00
VERIFICAR INTERVALO DE 11 HORAS ENTRE JORNADAS

Dia 23 SHAYENE 21T 12:00:00
Dia 24 SHAYENE 23M 06:30:00


In [ ]:
# Busca escalas dos semanais

print('ESCALAS DOS SEMANAIS SAINDO OU RETORNANDO DE FÉRIAS \n')
for i in range(len(df_escalas.index)):
  if len(df_escalas.iloc[i].unique()) > 2:
    if search('[X|Y]\d{2}', str(df_escalas.iloc[i])):
      print(df_escalas.iloc[i].unique())

ESCALAS DOS SEMANAIS SAINDO OU RETORNANDO DE FÉRIAS 

['DENIS' 'X04' 'D6' 'D2' 'PCA' 'FOL']
['DIEGO' 'FER' 'X11']
['FELIPE' 'X05' 'H6' 'C4']
['FREITAS' 'FOL' 'ASS' 'Y03' 'K7' 'K3']
['GARCIA' 'X08' 'C7' 'A2']
['GUILHERME' 'X07' 'I6' 'I4']
['INES MIRANDA' 'FER' 'X09']
['ISABEL' 'X02' 'FER' 'ASS']
['JORGE FILHO' 'R5' 'FER' 'Y05']
['MARIA ROSANE' 'NB' 'FOL' 'FER' 'PCA' 'Y06']
['RAQUEL' 'FOL' 'PCA' 'Y02' 'R5']
['VALDIR' 'X06' 'D7' 'C6' 'E3']
['VALERIA' 'FOL' 'PCA' 'X10']
['VINICIUS' 'L1' 'FER' 'PCA' 'Y04' 'L7']


In [ ]:
# Verificar quem trabalhou no dia anterior

ontem = data_atual.day - 1


realizado_dia = pd.read_excel(arquivo_recent,
                              sheet_name='DADOS',
                              usecols=list(range(24, 57)))

# Selecionar linhas de interesse
realizado_dia = realizado_dia[4:]
realizado_dia.columns = realizado_dia.loc[4]
realizado_dia.drop(4)

# Using set_index() method
realizado_dia = realizado_dia.set_index('MISSAO/DIA')
realizado_dia = realizado_dia[int(ontem)]
realizado_dia = realizado_dia.dropna()
realizado_dia = pd.DataFrame(realizado_dia)

# Filtrar "_" e "FOL"
mascara = realizado_dia[ontem].str.contains("_")==False
df = realizado_dia[mascara]
df = df.drop(index='FOL')
print('MISSÕES E OPERADORES DE ONTEM')
df



MISSÕES E OPERADORES DE ONTEM


,16
MISSAO/DIA,
01M,DA COSTA
02M,CARBONERA
03M,SILVANA
04M,CORREA
05M,CAMILOTTI
06M,VALDOIR
07M,TELES
08M,MIRIAN
09M,FURLAN
